## Imports, definitions, examples

In [1]:
import ipywidgets as widgets
import json
import io
import os
import sys
import copy
import shutil
import jinja2
import uncertainties
from types import FunctionType, UnionType, NoneType
from datetime import date
from dataclasses import dataclass

In [2]:
# Path constants:
JSON_FOLDER_PATH = "json" # Path to the JSON folder.
PDF_FOLDER_PATH = "reports" # Path to the PDF folder.
SOURCE_FOLDER_PATH = "source/" # Path to the folder with .py and .tex files.
TEMPORARY_NAME = "temp" # Used to create temporary names for JSON and PDF files (until the choosen widgets are filled).

# Constants for structuring JSON files:
FUNCTION_SECTION_KEY = "FUNCTIONS" # Key for function data.
WIDGET_SECTION_KEY = "WIDGETS"  # Key for widget data.
FIRST_SERIES_KEY = "first_series"   # Key for first series data (inside function data).
SECOND_SERIES_KEY = "second_series" # Key for second series data  (inside function data).
AVERAGE_SERIES_KEY = "average_series" # Key for average series data (inside function data).

In [3]:
[working_directory] = !pwd
sys.path.append(working_directory+"/"+SOURCE_FOLDER_PATH) # Allows importing from the source folder.
from defined_widgets import *
from defined_lab_functions import *

### Infrastructure functions

In [4]:
def get_current_pathname(filetype: str) -> str:
    """Returns up-to-date pathname for JSON or PDF file.
    If the widgets used to generate pathname are not filled yet, returns a temporary pathname.
    If these widgets are filled, returns pathname constructed using widget values.

    Args:
        filetype (str): Specifies the filetype to return pathname for. Possible values are "pdf" and "json".
    """
    assert filetype in ["pdf", "json"], "Supported filetypes are 'pdf' and 'json'."
    name = TEMPORARY_NAME
    characters_to_replace = """!@#$%^&*()_+=[]{}?<>~№.,;:"`' """ # |\/ are not replaced
    
    try:
        lot_widget_value = lot.value         # Get widget values (strings).
        id_widget_value = experiment_id.value 
        for character in characters_to_replace:
            lot_widget_value = lot_widget_value.replace(character, "")
            id_widget_value = id_widget_value.replace(character, "")
    
        if lot_widget_value != "" and id_widget_value != "":
            today = str(date.today()).replace("-", "")
            name = "{}_{}_{}".format(today, lot_widget_value, id_widget_value)  # Full name.
    finally:
        match filetype:
            case "json":
                return "{}/{}.json".format(JSON_FOLDER_PATH, name)
            case "pdf":
                return "{}/{}.pdf".format(PDF_FOLDER_PATH, name)

# # Usage example:
# get_current_pathname("json")
# # Now change values of lot and experiment_id widgets and try again.

In [5]:
def generate_touch_json() -> FunctionType:
    """Returns a touch_json function.
    Contains a closure with a nonlocal variable used as a container for the previous JSON pathname.
    """
    # Removes temporary JSON file on the initial function call:
    json_temporary_pathname = "{}/{}.json".format(JSON_FOLDER_PATH, TEMPORARY_NAME)
    if os.path.exists(json_temporary_pathname):
        os.remove(json_temporary_pathname)

    old_json_pathname = get_current_pathname("json") # Initial value for a nonlocal variable

    def touch_json():
        """Creates or renames JSON file if necessary.
        If JSON file with an up-to-date pathname already exists, this function does nothing.
        """
        nonlocal old_json_pathname
        novel_json_pathname = get_current_pathname("json")
        
        if os.path.exists(old_json_pathname) and os.path.exists(novel_json_pathname) and old_json_pathname != novel_json_pathname:
            shutil.move(novel_json_pathname, novel_json_pathname.removesuffix(".json") + "-CONFLICTING.json")
            os.rename(old_json_pathname, novel_json_pathname)
            
        # If JSON file with a novel pathname does not exist, but the old one does, then rename. Renaming is done to keep data.
        if os.path.exists(old_json_pathname) and not os.path.exists(novel_json_pathname):
            os.rename(old_json_pathname, novel_json_pathname)
            print("File {} renamed to {}".format(old_json_pathname, novel_json_pathname))

        # If JSON file does not exist, then create it (and folder if necessary):
        elif not os.path.exists(old_json_pathname) and not os.path.exists(novel_json_pathname):
            json_folder = os.path.dirname(novel_json_pathname)
            if not os.path.exists(json_folder):
                os.makedirs(json_folder)
                print("Folder {}/ created.".format(json_folder))
            with open(novel_json_pathname, "a") as file:
                empty_jdict = {
                    WIDGET_SECTION_KEY : {},
                    FUNCTION_SECTION_KEY: {
                        FIRST_SERIES_KEY: {},
                        SECOND_SERIES_KEY: {},
                        AVERAGE_SERIES_KEY: {},
                    }
                }
                json.dump(empty_jdict, file, indent=4, sort_keys=True)
                print("File {} created.".format(novel_json_pathname))

        old_json_pathname = novel_json_pathname # Save novel pathname into nonlocal variable
    return touch_json

touch_json = generate_touch_json()

def get_widget_dict() -> dict:
    """Returns a dict with all defined input widgets.
    The items of this dict are {widget_name: widget_object} pairs.
    Output widgets and collections of widgets (e.g. widget_output.Output) are not included.
    """
    dict_accumulator = {}
    for object_name in list(globals()):
        obj = globals().get(object_name)
        
        if isinstance(obj, widgets.Widget): # Check if the obj is a widget
            if not isinstance(obj,          # Check if the widget is not an output widget or a collection of other widgets.
                              (widgets.widget_box.HBox,
                               widgets.widget_box.VBox,
                               widgets.widget_selectioncontainer.Tab,
                               widgets.widget_string.Textarea,
                               widgets.widget_upload.FileUpload,
                               widgets.widget_output.Output)) and object_name[0] != "_":            
                dict_accumulator[object_name] = obj
    return dict_accumulator

# # Usage example:
# for name, widget in get_widget_dict().items():
#     display(widget)

# # Usage example:
# for name in get_widget_dict().keys():
#     print(name)    

In [6]:
def deserialize() -> dict:
    """Returns a dict with data read from JSON."""
    touch_json()
    json_pathname = get_current_pathname("json")
    with open(json_pathname, "r") as file:
        jdict = json.load(file)
    return jdict

# # Usage example:
# deserialize() # Get all data

# # Usage example:
# deserialize()[WIDGET_SECTION_KEY] # Get data for widgets

# # Usage example:
# deserialize()[FUNCTION_SECTION_KEY][SECOND_SERIES_KEY] # Get data for second series

In [7]:
def serialize(supplied_values: dict, section_key: str | None = None) -> dict:
    """Writes data to JSON file. Returns a dict with changed items (empty if no changes were made).
    
    Args:
        supplied_dict (dict): Contains items to write in JSON. 
        section_key (str): Section of JSON to write in. Supported values are WIDGET_SECTION_KEY, FUNCTION_SECTION_KEY, None. 
    """
    assert section_key in [None, WIDGET_SECTION_KEY, FUNCTION_SECTION_KEY], \
    "Supported section_key values are WIDGET_SECTION_KEY, FUNCTION_SECTION_KEY, None."
    jdict = deserialize()
    if section_key:
        section = jdict[section_key]
    else:
        section = jdict
    
    changes = {}
    for key, value in supplied_values.items():
        if value != section.get(key):  # If value is not the same, write new value.
            changes[key] = value  
    section.update(changes)
    
    json_pathname = get_current_pathname("json")
    with open(json_pathname, "w") as file:
        json.dump(jdict, file, indent=4, sort_keys=True)
    return changes

# # Usage example:
# serialize({"test_series":{"test_name":{"nominal_result":40}}}, section_key = FUNCTION_SECTION_KEY)

In [8]:
@dataclass
class DataForFunctionCall:
    """Container for all the information required to execute and log a function call.
    
    Fields:
        fn (FunctionType): Function to calculate, see defined_lab_functions.py for examples of such functions.
        args (list): # List of arguments for the fn. Argument could be widget object, or the result of a previous function call, i.e. tuple (fn, key).
        key (str): This key determines where in the JSON to write the results (to which series).
        decimal_places (None | int): The number of decimal places in the formatted representation of the result.
        mask (str): Allows to log results under a different name (mask). Mask must be unique. If None, fn.__name__ will be used to log results, otherwise a mask will be used.
        natural_name (None | str): Name to show when printing results.
        unit (None | str): Unit to show when printing results.
        method (None | str): Method to show when printing results.
    """
    fn: FunctionType 
    args: list 
    key: str
    decimal_places: None | int = None   # UnionType, which is None or int, default value is None.
    mask: str = None
    natural_name:  None | str = None
    unit: None | str = None
    method: None | str = None

In [9]:
def prepare_arguments(raw_args: list, jdict: dict, inverse_widget_dict: dict) -> list:
    """Parses a heterogeneous list of arguments. Returns a list of floats or integers or None's.
    
    Args:
        raw_args (list): Heterogeneous argument list to parse. This list can contain items like
            * widget;
            * str;
            * tuple (FunctionType, str2), representing the result of previous function call. Here str2 is interpreted as a series name;
            * tuple (str1, str2), representing the results of previous function calls. Here str1 is interpreted as a name, and str2 as a series name.
                
        jdict (dict): Data from JSON.
        inverse_widget_dict (dict): Reversed version of get_widget_dict() dict.
    """
    arg_accumulator = []
    for item in raw_args:
        try:
            match item:
                case widgets.Widget():  #Is item a widget? Then use widget's name as the key and read widget's value from JSON data.
                    arg_accumulator.append(jdict[WIDGET_SECTION_KEY][inverse_widget_dict[item]])
                case (FunctionType(), str()): # Is item a (function, str) tuple?
                    argument_function, argument_call_key = item
                    arg_accumulator.append(jdict[FUNCTION_SECTION_KEY][argument_call_key][argument_function.__name__]["nominal_result"])
                case (str(), str()): # Is item a (str, str) tuple? First str is expected to be a function mask, second str is expected to be a series key.
                    argument_function_mask, argument_call_key = item
                    arg_accumulator.append(jdict[FUNCTION_SECTION_KEY][argument_call_key][argument_function_mask]["nominal_result"])
                case str(): # Is item a string? A key from JSON root is expected 
                    arg_accumulator.append(jdict[item])
                case _:
                    raise Exception("Item {} has type {}, which is not supported.".format(item, type(item)))
        except KeyError: 
             arg_accumulator.append(None)
    return arg_accumulator

## Usage example:
# serialize({"test_series":{"test_name":{"nominal_result":40}}}, section_key = FUNCTION_SECTION_KEY)
# jdict = deserialize()
# inverse_widget_dict = {widget_object: name for name, widget_object in get_widget_dict().items()}
# prepare_arguments([mass_of_pycnometer_empty_S1, mass_of_pycnometer_with_water_S1, ("test_name", "test_series")], jdict, inverse_widget_dict)

In [10]:
def reduce(raw_number: float, decimal_places: int) -> str:
    """Returns a number represenatation for printing. Rounds number, but keeps trailing zeros."""
    formatter = '.{}f'.format(decimal_places)
    formatted_result = format(round(raw_number, decimal_places), formatter)        
    return formatted_result

## Usage example:
# reduce(3.14160001, 6)

In [11]:
warning_runes = {
            quantity_success_message: "✅",
            quantity_warning_message: "💧",
            success_message: "🟢",
            range_message: "🟡",
            repeatability_message: "🟠"
    }

def execute_function_call(call: DataForFunctionCall) -> str:
    """Executes one function call. If calculation result is novel, logs the result and return a report string."""
    function = call.fn
    arguments = call.args # The arguments must have already been prepared with prepare_arguments
    argcount = function.__code__.co_argcount # Required number of arguments for the function.
    variable_names = function.__code__.co_varnames
    if call.mask: # If there is a mask, function will be logged using mask.
        function_appearance = call.mask
    else:
        function_appearance = function.__name__
    
    assert argcount == len(arguments), "Argument list of {} should contain {} args!".format(function,  argcount)

    # Exception here is sometimes thrown in a normal program flow.
    try:
        returned_object = function(*arguments) # <-- FUNCTION CALL
    except (TypeError, ZeroDivisionError): # Arguments can be zero by default or None.
        return None
    
    if isinstance(returned_object, tuple): # Tuple is expected to be (result, warnings)
        result = returned_object[0]
        warnings = returned_object[1]
        assert isinstance(warnings, list)
    else:
        result = returned_object
        warnings = None
        
    match result:
        case uncertainties.core.Variable(): # Type of result is ufloat
            assert isinstance(call.decimal_places, int)
            nominal_result = result.nominal_value
            uncertainty_reduced = reduce(result.std_dev, call.decimal_places)
            result_reduced = reduce(result.nominal_value, call.decimal_places)
            formatted_result = "{} ± {}".format(result_reduced, uncertainty_reduced)
            uncertainty = result.std_dev

        case float() | int(): # Type of result is float or int.
            nominal_result = result
            if isinstance(call.decimal_places, int):
                formatted_result = reduce(result, call.decimal_places)
            else:
                formatted_result = None
            uncertainty = None

        case NoneType(): # The function called can itself return None.
            nominal_result = None
            formatted_result = None
            uncertainty = None

        case _:
            raise Exception("Unhandled result {} with type {}.".format(result, type(result)))

    calculation_result_dict = {
        "nominal_result": nominal_result,
        "formatted_result": formatted_result,
        "uncertainty": uncertainty,
        "arguments": dict(zip(variable_names, arguments)),
        "warnings": warnings,
        "decimal_places": call.decimal_places,
        "natural_name": call.natural_name,
        "unit": call.unit,
        "method": call.method,
    }

    jdict = deserialize()
    json_pathname = get_current_pathname("json")
    
    # Exception here is sometimes thrown in a normal program flow.
    try: # We try to read data for exactly the same call.
        assert jdict[FUNCTION_SECTION_KEY][call.key][function_appearance] == calculation_result_dict  
        return None 
    except Exception: # If exception occurs, the result is novel.
        jdict[FUNCTION_SECTION_KEY][call.key][function_appearance] = calculation_result_dict
        with open(json_pathname, "w") as file:
            json.dump(jdict, file, indent=4, sort_keys=True)
        
        # String formatting for printing to logs.
        if formatted_result:
            result = formatted_result
            
        warnings_str = ""
        if warnings:
            for warning in warnings:
                warnings_str += warning_runes[warning]
        return "{} ({}): {} ——► {} {}".format(function_appearance, call.key, arguments, result, warnings_str)

In [12]:
def update(call_objects: list[DataForFunctionCall]):
    """Creates a real-time update+log loop for all DataForFunctionCall objects listed.
    On each such update, the while True loop activates and several waves of call calculations can occur.
    
    Args:
        call_objects (list of DataForFunctionCall objects): list of calls to recalculate on each widget update.
    """
    inverse_widget_dict = {widget_object: name for name, widget_object in get_widget_dict().items()}
    
    def on_value_change(change):
        """This function is triggered on widget update.
        On each such update, the while True loop activates and several waves of calculations can occur.
        This function will be bound to widgets to keep track of their updates.
        
        Args:
            change (traitlets.utils.bunch.Bunch): Object with data about widget value change.
        """
        widget_instance = change["owner"] # Widget object that was updated
        widget_old_value = change["old"]  # Old value
        widget_new_value = change["new"]  # New value
        widget_name = inverse_widget_dict[widget_instance]

        # Serializing new widget value:
        if isinstance(widget_instance, widgets.widget_date.DatePicker):         # DatePicker creates non-serializable date objects, so we format them.
            serialize({widget_name:str(widget_new_value)}, WIDGET_SECTION_KEY)  # Example of reverse transformation: date.fromisoformat("2022-12-09")
        else:
            serialize({widget_name:widget_new_value}, WIDGET_SECTION_KEY) 
        print("Change in widget value:\n🌱 {}: {} ——► {}".format(widget_name, widget_old_value, widget_new_value))
        print("\n╔═════ Update loop starts ════╗")
    
        # The while True loop allows for multiple waves of updates after a single widget value change.
        while True:
            jdict = deserialize()
            reports = []  # Changes reports for this wave of updates.
            for call in call_objects: 
                prepared_arguments = prepare_arguments(call.args, jdict, inverse_widget_dict)
                if not all(prepared_arguments): # If not all arguments have been looked up for this function, move on to the next functions.
                    continue                    
                    
                call_copy = copy.copy(call) # Copy is made to avoid modification of the original call object.
                call_copy.args = prepared_arguments
                reports.append(execute_function_call(call_copy)) # <-- FUNCTION CALL OCCURS INSIDE

            if any (reports):
                print("Changes for updated functions:")
                for report in reports:
                    if report:
                        print("🍁 {}".format(report))
                print("\n")
            else: # If there are no reports, then this is the last wave of updates for the given widget value change.
                refresh_output_widgets()
                break # Break "while True" loop.
                
    # Binding an on_value_change function to all widgets in get_widget_dict():
    for w in get_widget_dict().values(): 
        w.observe(on_value_change, names='value')

### State loading

In [13]:
def load_state():
    if not json_uploader.value:
        raise Exception("Please select a JSON file.")
    if os.path.exists(get_current_pathname("json")): # If there is a JSON file, make a copy:
        shutil.copy(get_current_pathname("json"), get_current_pathname("json").removesuffix(".json") + "-DUMPED.json")
    
    uploaded_file = json_uploader.value[0]
    saved_data = json.load(io.BytesIO(uploaded_file.content))
    saved_widget_data = saved_data[WIDGET_SECTION_KEY]
    number_of_saved_widgets = len(saved_widget_data.keys())

    defined_widgets = get_widget_dict()
    number_of_loaded_widgets = 0
    for saved_widget_name, saved_widget_value in saved_widget_data.items(): 
        try: 
            widget_to_handle = defined_widgets[saved_widget_name]
            if isinstance(widget_to_handle, widgets.widget_date.DatePicker): # Special handling of date picker widgets is required because date objects are not serializable
                 widget_to_handle.value = date.fromisoformat(saved_widget_value) # Reverse transformation from (date string in iso format) to (date object).
            elif saved_widget_name == "experiment_id": # String are joined to prevent rewriting of existing JSON file.
                widget_to_handle.value = saved_widget_value + "-LOADED"  # Rewritting can happen if experiment_id and lot widgets and also date became the same as for loaded file.
            else:
                widget_to_handle.value = saved_widget_value
            number_of_loaded_widgets += 1
        except:
            print("The value of widget '{}', ({}) was not loaded into the notebook.".format(saved_widget_name, saved_widget_value))
    print("Widget values loaded from JSON: {}/{}.".format(number_of_loaded_widgets, number_of_saved_widgets))

json_uploader_emptyline = widgets.Label("\xa0")
json_uploader_info = widgets.Label("To load widget values, select a JSON file and load the state.")
json_uploader = widgets.FileUpload(
    accept = ".json",  # Accepted file extensions
    multiple = False,
    description = "Select JSON",
    tooltip='Select a JSON file.',
)

 # Separate button here is used (instead of observing events on json_uploader) because the button allows loading same state several times in a row. 
 # Observing events of json_uploader will not allow this feature: if you try to load the state from the same JSON file twice, it will do nothing.
 # One possible cause because there is no "change" of the json_uploader state when you load the same file twice.

json_load_button = widgets.Button(
    description="Load state",
    button_style="warning",
    tooltip='Load state from selected JSON file.',
)

json_load_button.on_click(lambda _: load_state())

### A simple example of defining widgets and functions, serializing and creating update loop

In [14]:
# # Widgets for testing:
# test_widget1 = widgets.IntSlider(
#     description="testing_value_1",
#     style=dict(description_width='initial'), layout=dict(width='400px'),
# )

# test_widget2 = widgets.IntSlider(
#     description="testing_value_2",
#     style=dict(description_width='initial'), layout=dict(width='400px'),
# )

# test_widget3 = widgets.IntSlider(
#     description="testing_value_3",
#     style=dict(description_width='initial'), layout=dict(width='400px'),
# )

# # Functions for testing:
# def test_sum(var_a, var_b, var_c):
#     return var_a + var_b + var_c

# def test_prod(var_a, var_b):
#     return var_a * var_b

# def test_power(var_a, var_b):
#     return var_a ** var_b

# # Serialization of arbitrary data
# serialize({
#     "test_const_1" : 30.6907,
#     "test_const_2" : 3,
# })

In [15]:
# for widget in [test_widget1, test_widget2, test_widget3]:
#     display(widget)

In [16]:
# # Updating functions
# update([
#     DataForFunctionCall(fn = test_sum, key = FIRST_SERIES_KEY, args = [test_widget1, test_widget2, test_widget3]),
#     DataForFunctionCall(fn = test_sum, key = SECOND_SERIES_KEY, args = [test_widget2, test_widget2, test_widget2]),
#     DataForFunctionCall(fn = test_power, key = FIRST_SERIES_KEY,  args = [test_widget3, "test_const_2"]), 
#     DataForFunctionCall(fn = test_prod, key = FIRST_SERIES_KEY,  args = [(test_sum, SECOND_SERIES_KEY), (test_power, FIRST_SERIES_KEY)]),
# ])

### Widget table layout and result printing 

In [17]:
def get_series_results(series_key: str) -> str:
    """Returns a string for printing function calculation results.
    
    Args:
        series_key: the key to specify the series to show results for.
    """
    output_string = ""
    triggered_warnings = set()

    data = deserialize()[FUNCTION_SECTION_KEY][series_key]
    for name, item in data.items():
        if item["formatted_result"]:
            result_to_display = item["formatted_result"]
        else:
            result_to_display = item["nominal_result"]
            
        warnings_runeword = ""
        if item["warnings"]:
            for warning in item["warnings"]:
                triggered_warnings.add(warning)
                warnings_runeword += warning_runes[warning]
        output_string += "{}: {} {}\n".format(name, result_to_display, warnings_runeword)
    return (output_string, triggered_warnings)

def refresh_output_widgets():
    series1_results, series1_warnings  = get_series_results(FIRST_SERIES_KEY)
    series2_results, series2_warnings = get_series_results(SECOND_SERIES_KEY)
    series_average_results, series_average_warnings = get_series_results(AVERAGE_SERIES_KEY)
    
    tab1_output.value = "SERIES 1 \n" + series1_results + "\nAVERAGES \n" + series_average_results
    tab1_warnings = series1_warnings | series_average_warnings
    for warning in tab1_warnings:
        tab1_output.value += "\n" + warning_runes[warning] + " " + str(warning)
            
    tab2_output.value = "SERIES 2 \n" + series2_results + "\nAVERAGES \n" + series_average_results
    tab2_warnings = series2_warnings | series_average_warnings
    for warning in tab2_warnings:
        tab2_output.value += "\n" + warning_runes[warning] + " " + str(warning)
        
def select_all():
    for widget in printing_widgets_to_functions_map.keys():
        widget.value = True

def create_workspace():
    """Creates a table of widgets."""
    found_widgets = get_widget_dict().items()
    
    series1_input_widgets = [widget_instance for (name, widget_instance) in found_widgets if "_S1" in name]
    series2_input_widgets = [widget_instance for (name, widget_instance) in found_widgets if "_S2" in name]
    print_widgets =         [widget_instance for (name, widget_instance) in found_widgets if "print" in name]
    
    info_input_widgets = [widget_instance for (name, widget_instance) in found_widgets
                          if widget_instance not in series1_input_widgets + series2_input_widgets + print_widgets + [tab1_output] + [tab2_output]]
    info_input_widgets.insert(21, json_uploader)

    info_box = widgets.VBox(info_input_widgets)
    series1_box = widgets.HBox([widgets.VBox(series1_input_widgets), tab1_box])
    series2_box = widgets.HBox([widgets.VBox(series2_input_widgets), tab2_box])
    print_box = widgets.VBox(print_widgets)
    
    tab = widgets.Tab()
    tab.children = [box for box in [info_box, series1_box, series2_box, print_box]]
    tab.titles = ["Info", "Series 1", "Series 2", "Create report"]

    print_pdf_button.on_click(lambda _: generate_pdf())
    print_select_all_button.on_click(lambda _: select_all())
    return tab

### PDF generation

In [18]:
# Widgets -> functions mapping (for printing results)
printing_widgets_to_functions_map = { 
print_average_alcohol_content_by_mass: average_alcohol_content_by_mass,
print_average_alcohol_content_by_volume: average_alcohol_content_by_volume,
print_average_real_extract: average_real_extract,
print_average_original_extract: average_original_extract,
print_average_apparent_extract: average_apparent_extract,
print_average_apparent_degree_of_fermentation: average_apparent_degree_of_fermentation,
print_average_real_degree_of_fermentation: average_real_degree_of_fermentation,
print_average_specific_gravity_of_beer: average_specific_gravity_of_beer,
print_average_original_gravity: average_original_gravity,
print_average_beer_pH: average_beer_pH
}

In [19]:
latex_jinja_env = jinja2.Environment(
    variable_start_string = '\VAR{',
    variable_end_string = '}',
    line_statement_prefix = '%%',
    line_comment_prefix = '%#',
    trim_blocks = True,
    autoescape = False,
    loader = jinja2.FileSystemLoader(os.path.abspath('.'))
)

def get_tags_with_uncertainty(): 
    output_list = []
    jdict = deserialize()[FUNCTION_SECTION_KEY][AVERAGE_SERIES_KEY]
    for name, data in jdict.items():
        if data["uncertainty"]:
            if data["natural_name"]:
                output_list.append(data["natural_name"])
            else:
                output_list.append(name)
    return output_list

def escape(string_to_escape: str) -> str: 
    """Escape special Latex characters."""
    if string_to_escape:
        replacements = {"%":"\%",
                        "$":"\$",
                        "{":"\{",
                        "}":"\}",
                        "_":"\_",
                        "&":"\&",
                        "#":"\#",
                        "~":"\~{}",
                        "^":"\^{}",
                        '"':'\\texttt{"}',
                       }
        for key, value in replacements.items():
            string_to_escape = string_to_escape.replace(key, value)
        return string_to_escape
    else:
        return ""

def generate_pdf():
    """Generates a PDF file on call."""
    current_day =  str(date.today()).replace("-", "")
    serialize({"current_day": current_day})
    
    function_data = deserialize()[FUNCTION_SECTION_KEY][AVERAGE_SERIES_KEY]
    inverse_tag_names = {widget.description:function.__name__ for widget, function in printing_widgets_to_functions_map.items()}
    
    requested_tags = []
    for widget in printing_widgets_to_functions_map.keys():
        if widget.value == True:
            requested_tags.append(widget.description)
    
    if not requested_tags:
        raise Exception("Please select the data to include in the report.")
        
    for tag in requested_tags:
        try:
            function_name = inverse_tag_names[tag]
            function_data[function_name]["formatted_result"]
        except KeyError:
            raise Exception("Value for '{} ({})' not found.".format(tag, function_name))

    tags_without_accreditation = [widget.description for widget in printing_widgets_to_functions_map.keys()] # Currently all methods are not accredited
    tags_with_uncertainty = get_tags_with_uncertainty()

    info_for_not_accredited = any(tag in tags_without_accreditation for tag in requested_tags)
    info_for_with_uncertainty = any(tag in tags_with_uncertainty for tag in requested_tags)
    tags_without_uncertainty = [tag for tag in requested_tags if tag not in tags_with_uncertainty]
    info_for_without_uncertainty = any(tag in tags_without_uncertainty for tag in requested_tags)

    template = latex_jinja_env.get_template(SOURCE_FOLDER_PATH+'jinja_template.tex')
    with open(SOURCE_FOLDER_PATH+"report.tex","w+") as file: # Creating .tex file with Jinja2
        file.write(template.render(
            escape = escape,
            requested_tags = requested_tags,
            widget = deserialize()[WIDGET_SECTION_KEY],
            current_day = current_day,
            function_data = function_data,
            inverse_tag_names = inverse_tag_names,
            
            tags_without_accreditation = tags_without_accreditation,
            tags_with_uncertainty = tags_with_uncertainty,
            tags_without_uncertainty = tags_without_uncertainty,
            
            # Bools: 
            info_for_not_accredited = info_for_not_accredited,
            info_for_with_uncertainty = info_for_with_uncertainty,
            info_for_without_uncertainty = info_for_without_uncertainty,
        ))
    
    # Call shell commands with ipython ✧ﾟ･:* magics *:･ﾟ✧ :
    !pdflatex -output-directory {SOURCE_FOLDER_PATH} report.tex # Generate PDF with pdflatex
    !pdflatex -output-directory {SOURCE_FOLDER_PATH} report.tex # Second pass
    !mkdir -p {PDF_FOLDER_PATH} #Create folder for PDF's if it does not exist
    !mv {SOURCE_FOLDER_PATH + "report.pdf"} {get_current_pathname("pdf")} #Rename PDF
    !open {get_current_pathname("pdf")} #Open PDF

### Main update call

In [20]:
update([
    DataForFunctionCall(fn = mass_of_sample_before_distillation, decimal_places = 2, key = FIRST_SERIES_KEY, args = [mass_of_distillation_flask_with_sample_S1, mass_of_distillation_flask_empty_S1]),
    DataForFunctionCall(fn = difference, mask = "mass_of_distillate", key = FIRST_SERIES_KEY, decimal_places = 2, args = [mass_of_receiver_flask_after_distillation_S1, mass_of_receiver_flask_empty_S1]),
    DataForFunctionCall(fn = difference, mask = "mass_of_residue_after_distillation", decimal_places = 2, key = FIRST_SERIES_KEY, args = [mass_of_distillation_flask_after_distillation_S1, mass_of_distillation_flask_empty_S1]),
    DataForFunctionCall(fn = specific_gravity_using_pycnometer, mask = "specific_gravity_of_distillate", decimal_places = 5, key = FIRST_SERIES_KEY, args = [mass_of_pycnometer_with_distillate_S1, mass_of_pycnometer_empty_S1, mass_of_pycnometer_with_water_S1]),
    DataForFunctionCall(fn = alcohol_content_by_mass, decimal_places = 2, key = FIRST_SERIES_KEY, args = [("specific_gravity_of_distillate", FIRST_SERIES_KEY)]),
    DataForFunctionCall(fn = specific_gravity_using_pycnometer, mask = "specific_gravity_of_beer", decimal_places = 5, key = FIRST_SERIES_KEY, args = [mass_of_pycnometer_with_beer_S1, mass_of_pycnometer_empty_S1, mass_of_pycnometer_with_water_S1]), ###
    DataForFunctionCall(fn = alcohol_content_by_volume, decimal_places = 2, key = FIRST_SERIES_KEY, args = [(alcohol_content_by_mass, FIRST_SERIES_KEY), ("specific_gravity_of_beer", FIRST_SERIES_KEY)]),
    DataForFunctionCall(fn = specific_gravity_using_pycnometer, mask = "specific_gravity_of_residue", decimal_places = 5, key = FIRST_SERIES_KEY, args = [mass_of_pycnometer_with_residue_S1, mass_of_pycnometer_empty_S1, mass_of_pycnometer_with_water_S1]),
    DataForFunctionCall(fn = extract, mask = "real_extract", decimal_places = 2, key = FIRST_SERIES_KEY, args = [("specific_gravity_of_residue", FIRST_SERIES_KEY)]),
    DataForFunctionCall(fn = extract, mask = "apparent_extract", decimal_places = 2, key = FIRST_SERIES_KEY, args = [("specific_gravity_of_beer", FIRST_SERIES_KEY)]),
    DataForFunctionCall(fn = original_extract, decimal_places = 2, key = FIRST_SERIES_KEY, args = [(alcohol_content_by_mass, FIRST_SERIES_KEY), ("real_extract", FIRST_SERIES_KEY)]),  
    DataForFunctionCall(fn = real_degree_of_fermentation, decimal_places = 1, key = FIRST_SERIES_KEY, args = [(alcohol_content_by_mass, FIRST_SERIES_KEY), ("real_extract", FIRST_SERIES_KEY)]),  
    DataForFunctionCall(fn = apparent_degree_of_fermentation, decimal_places = 1, key = FIRST_SERIES_KEY, args =  [(original_extract, FIRST_SERIES_KEY), ("apparent_extract", FIRST_SERIES_KEY)]),
    DataForFunctionCall(fn = spirit_indication, decimal_places = 1, key = FIRST_SERIES_KEY, args = [("specific_gravity_of_distillate", FIRST_SERIES_KEY)]),
    DataForFunctionCall(fn = degrees_of_gravity_lost, decimal_places = 1, key = FIRST_SERIES_KEY, args = [(spirit_indication, FIRST_SERIES_KEY)]),
    DataForFunctionCall(fn = residue_gravity, decimal_places = 1, key = FIRST_SERIES_KEY, args = [("specific_gravity_of_residue", FIRST_SERIES_KEY)]),
    DataForFunctionCall(fn = original_gravity, decimal_places = 1, key = FIRST_SERIES_KEY, args = [(degrees_of_gravity_lost, FIRST_SERIES_KEY), (residue_gravity, FIRST_SERIES_KEY)]),

    DataForFunctionCall(fn = mass_of_sample_before_distillation, decimal_places = 2, key = SECOND_SERIES_KEY, args = [mass_of_distillation_flask_with_sample_S2, mass_of_distillation_flask_empty_S2]),
    DataForFunctionCall(fn = difference, mask = "mass_of_distillate", decimal_places = 2, key = SECOND_SERIES_KEY, args = [mass_of_receiver_flask_after_distillation_S2, mass_of_receiver_flask_empty_S2]),
    DataForFunctionCall(fn = difference, mask = "mass_of_residue_after_distillation", decimal_places = 2, key = SECOND_SERIES_KEY, args =  [mass_of_distillation_flask_after_distillation_S2, mass_of_distillation_flask_empty_S2]),
    DataForFunctionCall(fn = specific_gravity_using_pycnometer, mask = "specific_gravity_of_distillate", decimal_places = 5, key = SECOND_SERIES_KEY, args = [mass_of_pycnometer_with_distillate_S2, mass_of_pycnometer_empty_S2, mass_of_pycnometer_with_water_S2]),
    DataForFunctionCall(fn = alcohol_content_by_mass, decimal_places = 2, key = SECOND_SERIES_KEY, args = [("specific_gravity_of_distillate", SECOND_SERIES_KEY)]),
    DataForFunctionCall(fn = specific_gravity_using_pycnometer, mask = "specific_gravity_of_beer", decimal_places = 5, key = SECOND_SERIES_KEY, args = [mass_of_pycnometer_with_beer_S2, mass_of_pycnometer_empty_S2, mass_of_pycnometer_with_water_S2]),
    DataForFunctionCall(fn = alcohol_content_by_volume, decimal_places = 2, key = SECOND_SERIES_KEY, args = [(alcohol_content_by_mass, SECOND_SERIES_KEY), ("specific_gravity_of_beer", SECOND_SERIES_KEY)]),
    DataForFunctionCall(fn = specific_gravity_using_pycnometer, mask = "specific_gravity_of_residue", decimal_places = 5, key = SECOND_SERIES_KEY, args = [mass_of_pycnometer_with_residue_S2, mass_of_pycnometer_empty_S2, mass_of_pycnometer_with_water_S2]),
    DataForFunctionCall(fn = extract, mask = "real_extract", decimal_places = 2, key = SECOND_SERIES_KEY, args = [("specific_gravity_of_residue", SECOND_SERIES_KEY)]),
    DataForFunctionCall(fn = extract, mask = "apparent_extract", decimal_places = 2, key = SECOND_SERIES_KEY, args = [("specific_gravity_of_beer", SECOND_SERIES_KEY)]),
    DataForFunctionCall(fn = original_extract, decimal_places = 2, key = SECOND_SERIES_KEY, args = [(alcohol_content_by_mass, SECOND_SERIES_KEY), ("real_extract", SECOND_SERIES_KEY)]),  
    DataForFunctionCall(fn = real_degree_of_fermentation, decimal_places = 1, key = SECOND_SERIES_KEY, args = [(alcohol_content_by_mass, SECOND_SERIES_KEY), ("real_extract", SECOND_SERIES_KEY)]),  
    DataForFunctionCall(fn = apparent_degree_of_fermentation, decimal_places = 1, key = SECOND_SERIES_KEY, args =  [(original_extract, SECOND_SERIES_KEY), ("apparent_extract", SECOND_SERIES_KEY)]),
    DataForFunctionCall(fn = spirit_indication, decimal_places = 1, key = SECOND_SERIES_KEY, args = [("specific_gravity_of_distillate", SECOND_SERIES_KEY)]),
    DataForFunctionCall(fn = degrees_of_gravity_lost, decimal_places = 1, key = SECOND_SERIES_KEY, args = [(spirit_indication, SECOND_SERIES_KEY)]),
    DataForFunctionCall(fn = residue_gravity, decimal_places = 1, key = SECOND_SERIES_KEY, args = [("specific_gravity_of_residue", SECOND_SERIES_KEY)]),
    DataForFunctionCall(fn = original_gravity, decimal_places = 1, key = SECOND_SERIES_KEY, args = [(degrees_of_gravity_lost, SECOND_SERIES_KEY), (residue_gravity, SECOND_SERIES_KEY)]),
    
    DataForFunctionCall(fn = average_alcohol_content_by_volume, decimal_places = 2, key = AVERAGE_SERIES_KEY, natural_name = print_average_alcohol_content_by_volume.description, unit="% (v/v)", method = "9.2.1 EBC", args = [(alcohol_content_by_volume, FIRST_SERIES_KEY), (alcohol_content_by_volume, SECOND_SERIES_KEY)]),
    DataForFunctionCall(fn = average_alcohol_content_by_mass, decimal_places = 2, key = AVERAGE_SERIES_KEY, natural_name = print_average_alcohol_content_by_mass.description, unit="% (m/m)", method = "9.2.1 EBC", args = [(alcohol_content_by_mass, FIRST_SERIES_KEY), (alcohol_content_by_mass, SECOND_SERIES_KEY)]),
    DataForFunctionCall(fn = average_specific_gravity_of_beer, decimal_places = 5, key = AVERAGE_SERIES_KEY, natural_name = print_average_specific_gravity_of_beer.description, unit="", method = "9.43.1 EBC",  args = [("specific_gravity_of_beer", FIRST_SERIES_KEY), ("specific_gravity_of_beer", SECOND_SERIES_KEY)]),
    DataForFunctionCall(fn = average_original_extract, decimal_places = 2, key = AVERAGE_SERIES_KEY, natural_name = print_average_original_extract.description, unit="% Plato", method = "9.4 EBC",  args = [(original_extract, FIRST_SERIES_KEY), (original_extract, SECOND_SERIES_KEY)]),  
    DataForFunctionCall(fn = average_real_extract, decimal_places = 2, key = AVERAGE_SERIES_KEY, natural_name = print_average_real_extract.description, unit="% Plato", method = "9.4 EBC",   args = [("real_extract", FIRST_SERIES_KEY), ("real_extract", SECOND_SERIES_KEY)]),  
    DataForFunctionCall(fn = average_apparent_extract, decimal_places = 2, key = AVERAGE_SERIES_KEY, natural_name = print_average_apparent_extract.description, unit="% Plato", method = "9.4 EBC",  args = [("apparent_extract", FIRST_SERIES_KEY), ("apparent_extract", SECOND_SERIES_KEY)]),  
    DataForFunctionCall(fn = average_real_degree_of_fermentation, decimal_places = 1, key = AVERAGE_SERIES_KEY, natural_name = print_average_real_degree_of_fermentation.description, unit="%", method = "9.5 EBC",  args = [(real_degree_of_fermentation, FIRST_SERIES_KEY), (real_degree_of_fermentation, SECOND_SERIES_KEY)]),  
    DataForFunctionCall(fn = average_apparent_degree_of_fermentation, decimal_places = 1, key = AVERAGE_SERIES_KEY, natural_name = print_average_apparent_degree_of_fermentation.description, unit="%", method = "Beer-6 ASBC", args = [(apparent_degree_of_fermentation, FIRST_SERIES_KEY), (apparent_degree_of_fermentation, SECOND_SERIES_KEY)]),
    DataForFunctionCall(fn = average_original_gravity, decimal_places = 1, key = AVERAGE_SERIES_KEY, natural_name = print_average_original_gravity.description, unit="°Sacch.", method = "9.4 EBC", args = [(original_gravity, FIRST_SERIES_KEY), (original_gravity, SECOND_SERIES_KEY)]),
    DataForFunctionCall(fn = average_beer_pH, decimal_places = 3, key = AVERAGE_SERIES_KEY, natural_name = print_average_beer_pH.description, unit="", method = "9.35 EBC", args = [beer_pH_S1, beer_pH_S2]),
])

## Workspace

In [21]:
create_workspace()